# Visualization of the science validation survey

For this notebook to work, please download the data files in the `../data` directory. Look at the README.md file for instructions on how to download them.

The final movie result from this directory will be in `../results` directory

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
import astropy
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

In [4]:

# Science Validation survey visualization

conn = sqlite3.connect('../data/sv_20250729.db')

df = pd.read_sql_query('SELECT * FROM observations', conn)

In [5]:
df.columns.values

array(['observationId', 'exposure_name', 'controller', 'day_obs',
       'seq_num', 'physical_filter', 'band', 'fieldRA', 'fieldDec',
       'rotSkyPos', 'azimuth_start', 'azimuth_end', 'azimuth',
       'altitude_start', 'altitude_end', 'altitude',
       'zenith_distance_start', 'zenith_distance_end', 'zenith_distance',
       'airmass', 'exp_midpt', 'exp_midpt_mjd', 'obs_start',
       'observationStartMJD', 'obs_end', 'obs_end_mjd',
       'visitExposureTime', 'shut_time', 'visitTime', 'group_id',
       'cur_index', 'max_index', 'img_type', 'emulated',
       'science_program', 'observation_reason', 'target_name', 'air_temp',
       'pressure', 'humidity', 'wind_speed', 'wind_dir', 'dimm_seeing',
       'focus_z', 'simulated', 'vignette', 'vignette_min',
       'scheduler_note', 's_region', 'can_see_sky', 'n_inputs',
       'pixel_scale_min', 'pixel_scale_max', 'pixel_scale_median',
       'astrom_offset_mean_min', 'astrom_offset_mean_max',
       'astrom_offset_mean_median', 'ast

In [6]:
df['simulated']

0        None
1        None
2        None
3        None
4        None
         ... 
44516    None
44517    None
44518    None
44519    None
44520    None
Name: simulated, Length: 44521, dtype: object

In [7]:
ra = df['fieldRA'].values
mask = ra>180
ra[mask] = ra[mask]-360
df['RA'] = -np.deg2rad(ra)

In [8]:
df['Dec'] = np.deg2rad(df['fieldDec'])

In [9]:
from astropy.time import Time

In [13]:
! rm -f ../results/sv-viz/*.png

t=0
n_obs_in_frame = 10
frame_num=0
while t < 4452:
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='mollweide')
    t_obs = df.loc[t, 'observationStartMJD']
    tim = Time(t_obs, format='mjd')
    utc = tim.to_datetime()
    ax.set_title(f'Time: {utc}')
    night = df.query(f'observationStartMJD <= {t_obs}')
    current_discoveries_plot = df.query(f'observationStartMJD < {t_obs}')
    current_discoveries_plot.plot.scatter('RA','Dec',ax=ax, c='gray', alpha=0.05)
    
    time = Time(df.loc[t, 'observationStartMJD'], format='mjd')
    print(time)
    sun = astropy.coordinates.get_body(body='sun', time=time)
    moon = astropy.coordinates.get_body(body='moon', time=time)
    print(sun)
    print(moon)
    
    obs = df.query(f'observationId >= {t} & observationId < {t+n_obs_in_frame}')
    
    u_obs = obs[obs['band'] == 'u']
    g_obs = obs[obs['band'] == 'g']
    r_obs = obs[obs['band'] == 'r']
    i_obs = obs[obs['band'] == 'i']
    z_obs = obs[obs['band'] == 'z']
    y_obs = obs[obs['band'] == 'y']
    
    u_obs.plot.scatter('RA','Dec',ax=ax, c='blue', label='u filter',alpha=0.5)
    g_obs.plot.scatter('RA','Dec',ax=ax, c='cyan', label='g filter',alpha=0.5)
    r_obs.plot.scatter('RA','Dec',ax=ax, c='green', label='r filter',alpha=0.5)
    i_obs.plot.scatter('RA','Dec',ax=ax, c='yellow', label='i filter',alpha=0.5)
    z_obs.plot.scatter('RA','Dec',ax=ax, c='red', label='z filter',alpha=0.5)
    y_obs.plot.scatter('RA','Dec',ax=ax, c='purple', label='y filter',alpha=0.5)

    ax.set_xticklabels(['150°','120°','90°','60°','30°','0°','-30°','-60°','-90°','-120°','-150°'])

    sun_ra = sun.ra.degree
    if sun_ra>180:
        sun_ra-=360

    moon_ra = moon.ra.degree
    if moon_ra>180:
        moon_ra-=360

    ax.scatter(-np.deg2rad(sun_ra),np.deg2rad(sun.dec.degree),c='y',edgecolor='k',s=40)
    ax.scatter(-np.deg2rad(moon_ra),np.deg2rad(moon.dec.degree),c='gray',edgecolor='k',s=40)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3, frameon=False)
    fig.savefig(f'../results/sv-viz/frame_{frame_num:05d}.png')
    plt.close()
    
    frame_num+=1
    t+=n_obs_in_frame

60847.14660269778
<SkyCoord (GCRS: obstime=60847.14660269778, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (89.64691205, 23.43557576, 1.01620657)>
<SkyCoord (GCRS: obstime=60847.14660269778, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (27.37623048, 14.64635507, 0.00244529)>
60847.151693167834
<SkyCoord (GCRS: obstime=60847.151693167834, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (89.65220797, 23.43558775, 1.01620693)>
<SkyCoord (GCRS: obstime=60847.151693167834, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (27.44560389, 14.67711796, 0.0024452)>
60847.15749001952
<SkyCoord (GCRS: obstime=60847.15749001952, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (89.65823878, 23.43560119, 1.01620735)>
<SkyCoord (GCRS: obstime=6084

In [14]:
framerate = 15

In [17]:
! cd ../results/sv-viz && ffmpeg -framerate {framerate} -i "frame_%05d.png" -c:v libx264 -pix_fmt yuv420p ../sv-viz.mp4

ffmpeg version 5.1.6 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 11 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-D_FORTIFY_SOURCE=2 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -march=x86-64-v2 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libvo-amrwbenc --enable-version3 --enable-bzlib --disable-crysta